### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

***Order Management Bot - Intelligent Order Management Assistant***

***Extract Order related information based on User Input Query***

In [16]:
#Setup the right Environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Force load .env from the current working directory
dotenv_path = "/home/datascience/ns-user-61/scripts/validation/ns-user-61.env"  # Ensure it's in the correct directory
load_dotenv(dotenv_path)

True

In [17]:
# Import the required libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import PyPDFDirectoryLoader

import os

In [18]:
import pandas as pd
# Importing the Order Data into Data Frame1

# Load the dataset
OrderDetails = pd.read_excel('Headers_Details_with_Customer.xlsx')
OrderDetails.head()

,HEADER_ID,ORDER_NUMBER,SOURCE_ORDER_ID,ORDERED_DATE,CUSTOMER_PO_NUMBER,ORDER_TYPE_CODE,TRANSACTIONAL_CURRENCY_CODE,STATUS_CODE,PARTIAL_SHIP_ALLOWED_FLAG,OPEN_FLAG,...,LAST_UPDATE_DATE,LAST_UPDATED_BY,PARTY_NAME,PARTY_NUMBER,CITY,COUNTRY,ADDRESS1,ADDRESS2,ADDRESS3,ADDRESS4
0,100143718873134,13005673,300046227964779,2021-10-20,F21062201500-OW-1,HARDWARE INVOICE ONLY,USD,DOO_DRAFT,Y,N,...,2025-02-26 16:53:00,mounika.seetha@oracle.com,ÂØåÂ£´ÈÄöÊ†™Âºè‰ºöÁ§æ,1260408,Â∑ùÂ¥éÂ∏Ç,JP,‰∏≠ÂéüÂå∫‰∏äÂ∞èÁî∞‰∏≠ÔºîÔºçÔºëÔºçÔºë,NaN,NaN,NaN
1,302944170908266,13051401,302944170908266,2025-02-10,NaN,ALL BUSINESS MODELS,AUD,OPEN,N,Y,...,2025-02-20 17:46:00,OAL-INTEGRATOR-USER_WW@ORACLE.COM,NSW Health Pathology,A5L8CQG,Newcastle,AU,"Level 5, 45 Watt St",NaN,NaN,NaN
2,302944172117270,13051402,302944172117270,2025-02-10,CC-5454,CLOUD SERVICES,USD,DOO_DRAFT,N,Y,...,2025-02-10 08:26:00,PRAVEEN.ENAMANDRAM@ORACLE.COM,"Peerless Construction, LLC",11935310,WEST BRANCH,US,PO Box 813,NaN,NaN,NaN
3,302944170908470,13051403,302944170908470,2025-02-10,NaN,CLOUD SERVICES,USD,OPEN,N,Y,...,2025-02-10 09:31:00,mounika.seetha@oracle.com,comexpextvalid@gmail.com-580268,A5X9WSP,Redwood City,US,200 Oracle Parkway,NaN,NaN,NaN
4,302944170908712,13051404,302944170908712,2025-02-10,NaN,HARDWARE,USD,OPEN,N,Y,...,2025-02-10 11:49:00,mounika.seetha@oracle.com,Success Foods Management Group LLC c/o Torchy'...,A57642C,Austin,US,4501 Springdale Rd,NaN,NaN,NaN


In [19]:
# Importing the Order Line Details Data into Data Frame 2
OrderLineDetails = pd.read_excel('Flines_Details.xlsx')
OrderLineDetails.head()

,HEADER_ID,SOURCE_ORDER_NUMBER,FULFILL_LINE_ID,ITEM_NUMBER,ORDERED_QTY,STATUS_CODE,SOURCE_LINE_NUMBER,REQUEST_ARRIVAL_DATE,REQUEST_SHIP_DATE,SCHEDULE_ARRIVAL_DATE,SCHEDULE_SHIP_DATE,EXTENDED_AMOUNT,UNIT_LIST_PRICE,UNIT_SELLING_PRICE,TRADE_COMPLIANCE_RESULT_CODE,TRADE_COMPLIANCE_DATE
0,100143718873134,13005673.0,1.001437e+14,B63536,1.0,CLOSED,1,2021-10-20 18:59:50,NaT,NaT,NaT,400,120,400,ORA_PASSED,2021-10-20 19:00:39
1,302944170908266,13051401.0,3.029442e+14,B102173,1.0,CLOSED,1,NaT,2025-02-10 04:49:04,NaT,NaT,750,750,750,ORA_PASSED,2025-02-20 17:46:43
2,302944170908266,13051401.0,3.029442e+14,B102174,1.0,CLOSED,2,NaT,2025-02-10 04:49:04,NaT,NaT,750,750,750,ORA_PASSED,2025-02-20 17:46:41
3,302944172117270,13051402.0,3.029442e+14,B81510,10.0,CREATED,1,NaT,2025-02-10 08:25:42,NaT,NaT,NaN,9.9900000000000005684341886080801486969E01,9.9900000000000005684341886080801486969E01,NaN,NaT
4,302944172117270,13051402.0,3.029442e+14,B108230,2000.0,CREATED,3,NaT,2025-02-10 08:25:42,NaT,NaT,NaN,9.9900000000000005684341886080801486969E01,9.9900000000000005684341886080801486969E01,NaN,NaT


In [20]:
# Treating the Missing Data
OrderDetails = OrderDetails.fillna('Information Not Available')
OrderLineDetails = OrderLineDetails.fillna('Information Not Available')

In [21]:
# Method to create the Order level context
def getOrderContext(OrderNumber):
    results_def = OrderDetails[OrderDetails['ORDER_NUMBER'] == OrderNumber]
    results_def = results_def.reset_index(drop=True)
    context_def = "ORDER_NUMBER :" + str(results_def['ORDER_NUMBER'][0]) + "\n" +"ORDER_TYPE_CODE :" + str(results_def['ORDER_TYPE_CODE'][0]) + "\n" + "STATUS_CODE :" + str(results_def['STATUS_CODE'][0]) + "\n" + "TRANSACTIONAL_CURRENCY_CODE :" + str(results_def['TRANSACTIONAL_CURRENCY_CODE'][0]) + "\n" + "CANCELED_FLAG :" + str(results_def['CANCELED_FLAG'][0]) + "\n" + "SUBMITTED_DATE :" + str(results_def['SUBMITTED_DATE'][0])+ "\n" + "PARTY_NAME :" + str(results_def['PARTY_NAME'][0]) + "\n" + "COUNTRY :" + str(results_def['COUNTRY'][0]) + "\n" + "ORDERED_DATE :" + str(results_def['ORDERED_DATE'][0]) + "\n" + "CUSTOMER_PO_NUMBER :" + str(results_def['CUSTOMER_PO_NUMBER'][0]) + "\n" 
    return context_def

In [22]:
# Method to create the Order lines level context
def getOrderlinesContext(OrderNumber_lines):
    line_results_def = OrderLineDetails[OrderLineDetails['SOURCE_ORDER_NUMBER'] == OrderNumber_lines]
    line_results_def = line_results_def.reset_index(drop=True)
    lines_context_def = ""
    for index_def, row_def in line_results_def.iterrows():
            value_ORDER_LINE_NUMBER = index_def + 1
            value_SOURCE_ORDER_NUMBER = row_def['SOURCE_ORDER_NUMBER']
            value_ITEM_NUMBER = row_def['ITEM_NUMBER']
            value_STATUS_CODE = row_def['STATUS_CODE']
            value_ORDERED_QTY = row_def['ORDERED_QTY']
            value_SCHEDULE_ARRIVAL_DATE = row_def['SCHEDULE_ARRIVAL_DATE']
            value_REQUEST_ARRIVAL_DATE = row_def['REQUEST_ARRIVAL_DATE']
            value_SOURCE_LINE_NUMBER = row_def['SOURCE_LINE_NUMBER']
            value_REQUEST_SHIP_DATE = row_def['REQUEST_SHIP_DATE']
            value_SCHEDULE_SHIP_DATE = row_def['SCHEDULE_SHIP_DATE']
            lines_context_def = lines_context_def + "\n" + f"{'Order line number'} : {value_ORDER_LINE_NUMBER} , {'SOURCE_ORDER_NUMBER'}: {value_SOURCE_ORDER_NUMBER} , {'ITEM_NUMBER'}: {value_ITEM_NUMBER} , {'STATUS_CODE'}:{value_STATUS_CODE}, {'ORDERED_QTY'}:{value_ORDERED_QTY}, {'SCHEDULE_ARRIVAL_DATE'}:{value_SCHEDULE_ARRIVAL_DATE}, {'REQUEST_ARRIVAL_DATE'}:{value_REQUEST_ARRIVAL_DATE}, {'SOURCE_LINE_NUMBER'}:{value_SOURCE_LINE_NUMBER}, {'REQUEST_SHIP_DATE'}:{value_REQUEST_SHIP_DATE}, {'SCHEDULE_SHIP_DATE'}:{value_SCHEDULE_SHIP_DATE}" 
    lines_context_def=lines_context_def.lstrip()
    return lines_context_def

In [23]:
# combining the Order Header level Context and Order Lines level Context
def createOrderContext(OrderNumber):
    # Get the information of  order from OrderDetails and Order line Details
    Entire_Order_Context_def = getOrderContext(OrderNumber) + getOrderlinesContext(OrderNumber)
    return Entire_Order_Context_def

In [24]:
# define Model
import oci
from langchain_community.chat_models import ChatOCIGenAI
from langchain_community.embeddings import OCIGenAIEmbeddings

In [30]:
# Defining the Model and Generating Chunks
# temperature = 0.5 - Balanced Response
# max-tokens = 4000 - Cap on the tokens that would be generated
def getOrderDetailsBot(Entire_Order_Context_input, query_input):
    model_def = ChatOCIGenAI(
        model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyanrlpnq5ybfu5hnzarg7jomak3q6kyhkzjsl4qj24fyoq",
        #model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyajqi26fkxly6qje5ysvezzrypapl7ujdnqfjq6hzo2loq",
        service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
        compartment_id="ocid1.compartment.oc1..aaaaaaaalsjh4sumix2khilco66ylipnlbjflpx6hocvkmtzn4i6r22y6awq",
        auth_profile="DEFAULT",
        auth_file_location='~/ns-user-61/.oci/config',
        provider="cohere",
        model_kwargs={"temperature": 0.2, "max_tokens": 4000, "top_p": 0.75,"top_k": 0, "frequency_penalty": 1}
        )
    llm_def = model_def
    #define Embedding
    embeddings_def = OCIGenAIEmbeddings(
        model_id = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyadcwe6y5jt655ox5hkm76vq2wz3hs7ljbfhljvithlnoq",
        service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
        compartment_id="ocid1.compartment.oc1..aaaaaaaalsjh4sumix2khilco66ylipnlbjflpx6hocvkmtzn4i6r22y6awq",
        auth_profile="DEFAULT",
        auth_file_location='~/ns-user-61/.oci/config'
        )

    # Create chunks for the context provided and convert into Embeddings using OCI Gen AI Embeddings 
    chunks_def = Entire_Order_Context_input.split('\n')
    vectorstore_def = FAISS.from_texts(texts = chunks_def,embedding=embeddings_def)
    vectorstore_retreiver_def = vectorstore_def.as_retriever(search_kwargs={"k": 10})
    keyword_retriever_def = BM25Retriever.from_texts(chunks_def)
    keyword_retriever_def.k =  10
    #Ensemble Retriever
    ensemble_retriever_def = EnsembleRetriever(retrievers=[vectorstore_retreiver_def,
                                                   keyword_retriever_def],
                                       weights=[0.5, 0.5])

    #Create prompt template

    template_def = """
    <|system|>
    You are a Order Management Assistant tasked with providing details related to a Order number in the **CONTEXT** and {query}
    Please provide a detailed description based on the query asked. Take reference of the Order Information provided in the Entire Order context to respond to the User's Query

    ### **Instructions:**
    - Use the provided **CONTEXT** to shape your response.
    - Be **professional**, while maintaining a **warm and engaging tone**.
    - Think **step by step** before formulating your answer.
    - Ensure the response is **persuasive, clear, and effective**.
    - Please respond in as **minimum tokens** as possible.
    - Validate if you can find Order Number details in the {query} and request for the Order Number if you dont find it.
    - DO NOT include the {context} in your response
    - Please be specific to the users {query}


    **CONTEXT:**  
    {context}  
    </s>  
    <|user|>
    {query}  
    </s>  
    <|assistant|>  
    """

    prompt_def = ChatPromptTemplate.from_template(template_def)
    output_parser_def = StrOutputParser()

    #Create chain
    chain_def = (
        {"context": keyword_retriever_def, "query": RunnablePassthrough()}
        | prompt_def
        | llm_def
        | output_parser_def
    )
    
    return chain_def.invoke(query_input)
    

In [31]:
import re
# Extract Order Number from the User Input
def extract_order_number(user_input1):
    """Extracts an order number from user input."""
    match = re.search(r'\b\d{6,10}\b', user_input1)  # Adjust the pattern based on your order number format
    return match.group() if match else None

In [32]:
# Method to Handle the query based on User Input and Order Number extracted
def handle_order_query(user_input2, order_num):
    """Main function to process order queries."""

    order_details = createOrderContext(order_num)
    response = getOrderDetailsBot(user_input2, order_details)
    return response

In [33]:
user_data = {}  # Dictionary to store user session data
# Invoke the Order Management Bot and start conversation
def omBot_Conversation():
    print("Chatbot: Hello! I'm OM-Bot. Please enter your Order Number along with your query.\nType 'bye' to exit.")

    while True:
        user_input = input("You: ").strip().lower()

        if user_input == "bye":
            print("Chatbot: Goodbye!")
            break

        order_number = extract_order_number(user_input)  # Extract order number
        
        if order_number:
            user_data["order_number"] = int(order_number)  # Store order number in session
        
        if "hello" in user_input:
            print("Chatbot: Hello there!")
        elif "good morning" in user_input or "good afternoon" in user_input or "good evening" in user_input:
            print("Chatbot: Greetings of the day! I hope you are doing great.")
        elif "how are you" in user_input:
            print("Chatbot: I'm doing well, thank you!")
        elif "what is your name" in user_input:
            print("Chatbot: My name is OM-Bot.")
        else:
            if "order status" in user_input or "where is my order" in user_input:
                if "order_number" in user_data:
                    bot_response = handle_order_query(f"Order {user_data['order_number']}")
                else:
                    bot_response = "I don't have your order number. Can you please provide it?"
            else:
                bot_response = handle_order_query(user_input, user_data["order_number"])
                
            print(bot_response)


In [37]:
omBot_Conversation()

Chatbot: Hello! I'm OM-Bot. Please enter your Order Number along with your query.
Type 'bye' to exit.


You:  Helloo


Chatbot: Hello there!


You:  Get details of the order 13051402?


I have found the order details for order number 13051402. Here is a summary:

- Order Type: Cloud Services
- Status: DOO_DRAFT
- Currency: USD
- Customer: Peerless Construction, LLC
- Order Date: 2025-02-10
- Customer PO Number: CC-5454

The order contains three items:
1. Item B81510: 10 units, status CREATED.
2. Item B108230: 2000 units, status CREATED.
3. Item B109229: 2 units, status CREATED.

All items have a requested ship date of 2025-02-10 08:25:42, and the arrival dates are not available.

Is there anything else I can help you with regarding this order?


...

I have found the order number details you requested. Here is the information:

**Order Number:** 13051519
**Order Type Code:** ALL BUSINESS MODELS
**Status Code:** OPEN
**Transactional Currency Code:** USD
**Canceled Flag:** N
**Submitted Date:** 2025-02-19 08:07:00
**Party Name:** CommonSpirit Health
**Country:** US
**Ordered Date:** 2025-02-19 00:00:00
**Customer PO Number:** 123456

**Order Line Details:**
- Line 1: ITEM_NUMBER: 7607542, STATUS_CODE: AWAIT_SHIP, ORDERED_QTY: 1.0, SCHEDULE_ARRIVAL_DATE: 2025-02-21 13:31:00, REQUEST_ARRIVAL_DATE: 2025-02-19 08:07:04, REQUEST_SHIP_DATE: Information Not Available, SCHEDULE_SHIP_DATE: 2025-02-20 10:31:00
- Line 2: ITEM_NUMBER: 8218553, STATUS_CODE: AWAIT_SHIP, ORDERED_QTY: 1.0, SCHEDULE_ARRIVAL_DATE: 2025-02-21 13:31:00, REQUEST_ARRIVAL_DATE: 2025-02-19 08:07:04, REQUEST_SHIP_DATE: Information Not Available, SCHEDULE_SHIP_DATE: 2025-02-20 10:31:00
- Line 3: ITEM_NUMBER: 7607548, STATUS_CODE: AWAIT_SHIP, ORDERED_QTY: 1.0, SCHEDULE_ARRIVAL

You:  who is the customer on the order 13051404?


The customer for order number 13051404 is Success Foods Management Group LLC c/o Torchy's Taco.


You:  bye


Chatbot: Goodbye!
